In [13]:
import matplotlib.pyplot as plt
import pandas as pd
import os,glob
import warnings
import numpy as np
import seaborn
from numpy import linalg as L
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from impyute.imputation.cs import mice

In [2]:
data = pd.read_csv("测试异常值.csv", encoding="gbk")

In [3]:
data

,time,area,二产用电量,全社会用电量,一产用电量
0,20190401,杭州市区,28286922.0,53564762.0,12019.0
1,20190402,杭州市区,28501960.0,53628989.0,14025.0
2,20190403,杭州市区,28468456.0,53270758.0,14244.0
3,20190404,杭州市区,23653346.0,48242787.0,16126.0
4,20190405,杭州市区,24850084.0,46256790.0,9751.0
5,20190406,杭州市区,25046528.0,46245993.0,9715.0
6,20190407,杭州市区,25045151.0,46609235.0,10822.0
7,20190408,杭州市区,24011726.0,48138421.0,11369.0
8,20190409,杭州市区,NaN,48405854.0,11936.0
9,20190410,杭州市区,24082199.0,47586488.0,12562.0


In [4]:
data=data.drop('time',1)
data=data.drop('area',1)

## 均值, 重数, 中位数填补

In [5]:
#均值
def mean_interpolate(df):
    df_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    df = df_mean.fit_transform(df)
    return df

#中位数
def median_interpolate(df):
    df_median = SimpleImputer(missing_values=np.nan, strategy='median')
    df = df_median.fit_transform(df)
    return df

#重数
def most_frequent_interpolate(df):
    df_most_frequen = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    df = df_most_frequent.fit_transform(df)
    return df

In [6]:
pd.DataFrame(median_interpolate(data).round(0))

,0,1,2
0,28286922.0,53564762.0,12019.0
1,28501960.0,53628989.0,14025.0
2,28468456.0,53270758.0,14244.0
3,23653346.0,48242787.0,16126.0
4,24850084.0,46256790.0,9751.0
5,25046528.0,46245993.0,9715.0
6,25045151.0,46609235.0,10822.0
7,24011726.0,48138421.0,11369.0
8,25046528.0,48405854.0,11936.0
9,24082199.0,47586488.0,12562.0


## KNN填补

In [7]:
def knn_interpolate(df):
    imputer = KNNImputer(n_neighbors=4) 
    df = imputer.fit_transform(df) 
    return df

In [8]:
pd.DataFrame(knn_interpolate(data).round(0))

,0,1,2
0,28286922.0,53564762.0,12019.0
1,28501960.0,53628989.0,14025.0
2,28468456.0,53270758.0,14244.0
3,23653346.0,48242787.0,16126.0
4,24850084.0,46256790.0,9751.0
5,25046528.0,46245993.0,9715.0
6,25045151.0,46609235.0,10822.0
7,24011726.0,48138421.0,11369.0
8,24582376.0,48405854.0,11936.0
9,24082199.0,47586488.0,12562.0


## 多重插补

In [15]:
def iter_interpolate(df):
    df = mice(df.values)
    return df

In [16]:
pd.DataFrame(iter_interpolate(data).round(0))

,0,1,2
0,28286922.0,53564762.0,12019.0
1,28501960.0,53628989.0,14025.0
2,28468456.0,53270758.0,14244.0
3,23653346.0,48242787.0,16126.0
4,24850084.0,46256790.0,9751.0
5,25046528.0,46245993.0,9715.0
6,25045151.0,46609235.0,10822.0
7,24011726.0,48138421.0,11369.0
8,24772263.0,48405854.0,11936.0
9,24082199.0,47586488.0,12562.0
